In [6]:
import pandas as pd
import pathlib
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport
from xgboost import XGBRegressor
!ls -alrt "/Users/soumensardar/Library/CloudStorage/GoogleDrive-soumensardarintmain@gmail.com/.shortcut-targets-by-id/1zlA8VX-l_IufCZSfJIfxohpsEGrcLx4-/Taniya Paul Thesis paper/writing/Research Proposal/dataset"
!ls -alrt "/Users/soumensardar/Library/CloudStorage/GoogleDrive-soumensardarintmain@gmail.com/.shortcut-targets-by-id/1zlA8VX-l_IufCZSfJIfxohpsEGrcLx4-/Taniya Paul Thesis paper/writing/Research Proposal/dataset/Food Demand Dataset"
root_dir = pathlib.Path("/Users/soumensardar/Library/CloudStorage/GoogleDrive-soumensardarintmain@gmail.com/.shortcut-targets-by-id/1zlA8VX-l_IufCZSfJIfxohpsEGrcLx4-/Taniya Paul Thesis paper/writing/Research Proposal/dataset")

total 305296
-rw-rw-r--@  1 soumensardar  staff  35703675 Mar 27 23:19 LivestockMeat_Imports.csv
-rw-rw-r--@  1 soumensardar  staff  51253380 Mar 27 23:20 Forecasts for Product Demand-base.csv
-rw-------@  1 soumensardar  staff  45622278 Mar 27 23:22 online_retail_II.xlsx
-rwx------@  1 soumensardar  staff  23715344 Mar 27 23:22 Online Retail.xlsx
drwx------@  5 soumensardar  staff       160 Mar 27 23:25 Food Demand Dataset
-rw-r--r--@  1 soumensardar  staff      6148 Mar 30 12:10 .DS_Store
drwx------@ 14 soumensardar  staff       448 Mar 30 12:43 ..
drwxr-xr-x@  8 soumensardar  staff       256 Mar 31 11:23 .
total 36608
-rw-rw-r--@ 1 soumensardar  staff  18733819 Mar 27 23:25 train.csv
drwx------@ 5 soumensardar  staff       160 Mar 27 23:25 .
drwxr-xr-x@ 6 soumensardar  staff       192 Mar 27 23:25 notrequired
-rw-r--r--@ 1 soumensardar  staff      6148 Mar 30 12:10 .DS_Store
drwxr-xr-x@ 8 soumensardar  staff       256 Mar 31 11:23 ..


In [5]:
def get_dataset(df, target_col, test_size=.2):
    y = df[target_col]
    X = df.drop(columns=[target_col])
    return train_test_split(X, y, test_size=test_size)

In [ ]:
df = pd.read_csv(root_dir /"Forecasts for Product Demand-base.csv")
X_train, X_test, y_train, y_test = get_dataset(df, "Order_Demand", .2)


# create model instance
bst = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')
# fit model
bst.fit(X_train, y_train)
# make predictions
preds = bst.predict(X_test)